<a href="https://colab.research.google.com/github/TomKellyGenetics/toy_snp_caller/blob/master/Copy_of_PyCUDA_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Part 1**
First of all you need to check whether you have a GPU-accelerated Runtime. To do this, Select the 'Runtime' menu and then choose 'Change runtime type'. You should then choose 'GPU' if it is not already selected. Once done, you may need to select the 'Connect' option at the top right of your notebook window. 
When it has loaded and connected you should run the following command to check whether the system is indeed using a GPU:

In [1]:
!nvidia-smi

Thu Oct 24 02:14:38 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

If all is well, you should see some output that includes the type of GPU being used (k80) and lists any processed running (none for the moment).

Next, we need to install PyCUDA:

In [2]:
pip install pycuda

     |████████████████████████████████| 1.6MB 3.5MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 471kB 42.0MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4535904 sha256=87b234838308f641ed6e1ac9bb255194141f4fd08268c882c1e827a0eb4ee421
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2019.1.1-py2.py3-none-any.whl size=58424 sha256=c8a1a926035866f7cb73c4099472bba4ad805ce4f1ccce44fcdc275b609ef9cd
  Stored in directory: /root/.cache/pip/wheels/83/df/0b/75ac4572aaa93e3eba6a58472635d0fda907f5f4cf884a3a0c
  Created wheel for mako: filename=Mako-1.1.0-cp36-none-any.whl size=75363 sha256=43435dd20a0cd1053a03048a2f517ec79bb7ea15da51a3080ef113ebb9460dc4
  Stored in directory: /root/.cache/pip/wheels/98/32/7b/a291926643fc1d1e02593e0d9e247c5a866a366b8343b7aa27
Successfully built pycuda pytools mako


Now we should be able to import the PyCUDA libraries that we need:

In [0]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

We will be using numpy quite a lot so let's import that right away:

In [0]:
import numpy as np

Okay, let's create a 4 x 4 array using numpy's random function and show the contents.

In [5]:
a = np.random.randn(4,4)
a

array([[-1.24375605, -2.05606006, -1.05551482, -1.87345212],
       [-0.50008479,  0.81762928, -0.79785209,  0.5618535 ],
       [ 1.20853345, -0.38096592, -0.94690202,  0.06241843],
       [-0.71626338, -0.2208585 , -1.57586999,  0.05438271]])

Let's check the type of one of the elements:

In [6]:
type(a[0,0])

numpy.float64

As you should see, this is of type float64. This is not always the right data type for kernel functions for the following reasons:
Only Enterprise GPUs (i.e. Volta/Pascal rather than GeForce/RTX) tend to support Full Precision (float64). Even then, they tend to have better support for lower precision (e.g. int32, fp16 etc.)
So it's better to either specify the type or cast it before passing it to the GPU:

In [52]:
b = np.random.randn(4,4)
b = b.astype(np.float32)
print(b)
print(type(b[0,0]))
print("")
c = np.random.randint(low=1, high=100, size=16, dtype=np.int16).reshape(4,4)
print(c)
print(type(c[0,0]))


[[ 0.37104166 -0.13156942  0.9888739   0.43702406]
 [ 0.17276785 -0.66560954  1.226529   -0.3124692 ]
 [ 0.5235419   3.1243346   0.66348433 -0.8306269 ]
 [-0.909838   -1.725645    0.34903932 -0.23539452]]
<class 'numpy.float32'>

[[17 60 54  8]
 [52 68 50 90]
 [26 10 56 63]
 [25 31 28 16]]
<class 'numpy.int16'>


Okay, now let's take a look at the way our data can be transferred over to the device (GPU) for a kernel to act upon.
In this example, we use the cuda.mem_alloc to allocate some gpu memory. In order to do this, we need to determine how much memory we need. For this we use the nbytes property of our variable.


In [64]:
b_gpu = cuda.mem_alloc(b.nbytes)
print(str(b.nbytes) + " bytes allocated")

64 bytes allocated


That has allocated the memory, so now we can transfer the data to the gpu memory (htod = host to device):

In [65]:
cuda.memcpy_htod(b_gpu, b)
print(str(b.nbytes) + " allocated to GPU: " + str(b_gpu.__sizeof__()))

64 allocated to GPU: 96


Next we need to create some code to execute on the GPU - our kernel function - that will operate on the data we just passed.

In this case all we are going to do is multiply the contents of the a matrix by 2.

In [66]:
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    // define index (flatten 4x4 matrix) to 1-dimension array (1x16 vector)
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)
print("kernel successfully initiated")

kernel successfully initiated


If there were no errors then our kernel has compiled and is ready to be called.

In [0]:
#assign the C function "doublify" as a python function "func" and run as func on GPU
func = mod.get_function("doublify")
func(b_gpu, block=(4,4,1))

Let's just take a look at what we just executed.
First we got a handle to the doublify function that we just created, then we called the function passing a_gpu as the function argument and we specified the launch configuration using the block parameter - more on this shortly.


In [68]:
b_doubled = np.empty_like(b)
cuda.memcpy_dtoh(b_doubled, b_gpu)
print(b_doubled)
print(b)

[[ 0.7420833  -0.26313883  1.9777478   0.8740481 ]
 [ 0.3455357  -1.3312191   2.453058   -0.6249384 ]
 [ 1.0470839   6.248669    1.3269687  -1.6612538 ]
 [-1.819676   -3.45129     0.69807863 -0.47078905]]
[[ 0.37104166 -0.13156942  0.9888739   0.43702406]
 [ 0.17276785 -0.66560954  1.226529   -0.3124692 ]
 [ 0.5235419   3.1243346   0.66348433 -0.8306269 ]
 [-0.909838   -1.725645    0.34903932 -0.23539452]]


Here you can see that every element of the matrix has been doubled

In [69]:
b_doubled / b

array([[2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.]], dtype=float32)

In [70]:
print(b[1,1]) 
print(b_doubled[1,1])

-0.66560954
-1.3312191


Okay, so that worked (hopefully) but what did the kernel code actually do? 
To more fully understand this we need to take a look at the way that kernels are launched (Slides will be presented).

**Part 2**

To better understand what is going on when you launch a kernel let's try a few experiments.

We'll create a simple kernel that just sets the elements of the input array to the thread index. This should help show what is going on


In [0]:
mod2 = SourceModule("""
  __global__ void getInfo(int *a)
  {
    int idx = threadIdx.x;
    a[idx] = threadIdx.x;
  }
  """)

In [72]:
# Create an empty 1d array
d = np.zeros((16), dtype=np.uint32)
# Allocate GPU memory
d_gpu = cuda.mem_alloc(d.nbytes)
# clear the gpu memory
cuda.memcpy_htod(d_gpu, d)

# Get the function handle
func2 = mod2.get_function("getInfo")
# Create a lauch configuration with 4 threads and 1 block
func2(d_gpu, block=(4,1,1))

# Create array for output
d_out = np.empty_like(d)
# Copy the gpu memory to the host
cuda.memcpy_dtoh(d_out, d_gpu)

# Print data out
print(d_out)

# Free up the memory
d_gpu.free()

[0 1 2 3 0 0 0 0 0 0 0 0 0 0 0 0]


Do you understand what the output has produced? 
Our kernel has launched a block with 4 threads (block = (4,1,1) means that 4 threads are created in the block on the x dimension, with no additional threads on the y or z dimensions). Each thread has written its threadIdx.x value into the array element corresponding to this value. Since there are only 4 threads, only the first 4 elements have been updated.
If we want to write into the other elements we will have to do one of several things:

*   Increase the number of threads in the block
*   Increase the number of blocks
*   Iterate within the kernel

Let's try these one by one.

By changing the number of threads per block to 16 (in the x dimension) we can fully populate the array.

In [90]:
# Allocate GPU memory
d_gpu = cuda.mem_alloc(d.nbytes)
# clear the gpu memory
cuda.memcpy_htod(d_gpu, d)

# Create a lauch configuration with 16 threads
func2(d_gpu, block=(16,1,1))

# Copy the gpu memory to the host
cuda.memcpy_dtoh(d_out, d_gpu)

# Present data as a matrix
print(d_out)

# Free gpu memory
d_gpu.free()

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]


In order to make increasing the number of blocks effective we will need to change the kernel so that it can use them. Without doing this each block will simply write into the same array element as the others. By adding the blockIdx.x into the kernel we can use it to map each thread and block to a specific element.

In [0]:
mod3 = SourceModule("""
  __global__ void getInfo(int *a)
  {
    // define index so that each block processes a separate thread
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    a[idx] = idx;
  }
  """)

In [82]:
# Allocate GPU memory
d_gpu = cuda.mem_alloc(d.nbytes)
# clear the gpu memory
cuda.memcpy_htod(d_gpu, d)

# Get the function handle
func3 = mod3.get_function("getInfo")
# Create a lauch configuration with 4 threads and 4 blocks
func3(d_gpu, block=(4,1,1), grid=(4,1,1))

# Copy the gpu memory to the host
cuda.memcpy_dtoh(d_out, d_gpu)

# Present data as a matrix
print(d_out)

# Free gpu memory
d_gpu.free()

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]


We can see that these components are important to assign a thread index to each block

In [83]:
mod3 = SourceModule("""
  __global__ void getInfo(int *a)
  {
    // define index so that each block processes a separate thread
    int idx = threadIdx.x * blockDim.x;
    a[idx] = idx;
  }
  """)
# Allocate GPU memory
d_gpu = cuda.mem_alloc(d.nbytes)
# clear the gpu memory
cuda.memcpy_htod(d_gpu, d)

# Get the function handle
func3 = mod3.get_function("getInfo")
# Create a lauch configuration with 4 threads and 4 blocks
func3(d_gpu, block=(4,1,1), grid=(4,1,1))

# Copy the gpu memory to the host
cuda.memcpy_dtoh(d_out, d_gpu)

# Present data as a matrix
print(d_out)

# Free gpu memory
d_gpu.free()

[ 0  0  0  0  4  0  0  0  8  0  0  0 12  0  0  0]


In [84]:
mod3 = SourceModule("""
  __global__ void getInfo(int *a)
  {
    # define index so that each block processes a separate thread
    int idx = threadIdx.x * blockIdx.x;
    a[idx] = idx;
  }
  """)
# Allocate GPU memory
d_gpu = cuda.mem_alloc(d.nbytes)
# clear the gpu memory
cuda.memcpy_htod(d_gpu, d)

# Get the function handle
func3 = mod3.get_function("getInfo")
# Create a lauch configuration with 4 threads and 4 blocks
func3(d_gpu, block=(4,1,1), grid=(4,1,1))

# Copy the gpu memory to the host
cuda.memcpy_dtoh(d_out, d_gpu)

# Present data as a matrix
print(d_out)

# Free gpu memory
d_gpu.free()

[0 1 2 3 4 0 6 0 0 9 0 0 0 0 0 0]


We miust also ensure that the block/grid configuration has enough spaces for all threads in the index

In [87]:
mod3 = SourceModule("""
  __global__ void getInfo(int *a)
  {
    // define index so that each block processes a separate thread
    int idx = threadIdx.x * blockIdx.x;
    a[idx] = idx;
  }
  """)
# Allocate GPU memory
d_gpu = cuda.mem_alloc(d.nbytes)
# clear the gpu memory
cuda.memcpy_htod(d_gpu, d)

# Get the function handle
func3 = mod3.get_function("getInfo")
# Create a lauch configuration with 4 threads and 4 blocks
func3(d_gpu, block=(4,1,1), grid=(5,1,1))

# Copy the gpu memory to the host
cuda.memcpy_dtoh(d_out, d_gpu)

# Present data as a matrix
print(d_out)

# Free gpu memory
d_gpu.free()

[ 0  1  2  3  4  0  6  0  8  9  0  0 12  0  0  0]


Lets try some iteration within the kernel next. This is only really used when the size of the problem exceeds the total number of threads available on the gpu but, for the sake of illustration we'll try it here.

This runs indexes in a loop on a smaller block than the number of indexes

In [0]:
mod4 = SourceModule("""
  __global__ void getInfo(int *a)
  {
    int idx = threadIdx.x;
    for (int i=0;i<16;i+=4)
    {
      a[idx + i] = idx + i;
    }
  }
  """)

In [92]:
# Allocate GPU memory
d_gpu = cuda.mem_alloc(d.nbytes)
# clear the gpu memory
cuda.memcpy_htod(d_gpu, d)

# Get the function handle
func3 = mod4.get_function("getInfo")
# Create a lauch configuration with 4 threads and 4 blocks
func3(d_gpu, block=(4,1,1))

# Copy the gpu memory to the host
cuda.memcpy_dtoh(d_out, d_gpu)

# Present data as a matrix
print(d_out)

# Free gpu memory
d_gpu.free()

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]


Okay, so as a final exercise, let's see if the following makes sense. Note that we are using a 3D matrix as input this time. However, as far as the C-based kernel is concerned, there is no difference between a 3D array and a 1D array - ultimately they are both just contiguous memory.
Check that you understand the output. 

In [0]:
mod5 = SourceModule("""
  __global__ void getInfo(int *a)
  {
    int idx = 4 * (blockIdx.x * blockDim.x + threadIdx.x);
    a[idx] = threadIdx.x;
    a[idx + 1] = blockIdx.x;
    a[idx + 2] = blockDim.x;
    a[idx + 3] = gridDim.x;
  }
  """)

In [94]:
# Create an empty 3d array
b = np.zeros((4,4,4), dtype=np.uint32)
# Allocate GPU memory
b_gpu = cuda.mem_alloc(b.nbytes)

# Get the function handle
func = mod5.get_function("getInfo")
# Create a lauch configuration with 4 threads and 4 blocks
func(b_gpu, block=(4,1,1), grid=(4,1,1))

# Create array for output
b_out = np.empty_like(b)
# Copy the gpu memory to the host
cuda.memcpy_dtoh(b_out, b_gpu)

# Present data as a matrix
print(b_out.reshape((4,4,4)))

[[[0 0 4 4]
  [1 0 4 4]
  [2 0 4 4]
  [3 0 4 4]]

 [[0 1 4 4]
  [1 1 4 4]
  [2 1 4 4]
  [3 1 4 4]]

 [[0 2 4 4]
  [1 2 4 4]
  [2 2 4 4]
  [3 2 4 4]]

 [[0 3 4 4]
  [1 3 4 4]
  [2 3 4 4]
  [3 3 4 4]]]


Try changing some of the parameters such as block size and see what happens

In [100]:
# Create an empty 3d array
b = np.zeros((6,3,4), dtype=np.uint32)
# Allocate GPU memory
b_gpu = cuda.mem_alloc(b.nbytes)

# Get the function handle
func = mod5.get_function("getInfo")
# Create a lauch configuration with 4 threads and 4 blocks
func(b_gpu, block=(2,2,1), grid=(6,3,1))

# Create array for output
b_out = np.empty_like(b)
# Copy the gpu memory to the host
cuda.memcpy_dtoh(b_out, b_gpu)

# Present data as a matrix
print(b_out.reshape((6,3,4)))


[[[0 0 2 6]
  [1 0 2 6]
  [0 1 2 6]]

 [[1 1 2 6]
  [0 2 2 6]
  [1 2 2 6]]

 [[0 3 2 6]
  [1 3 2 6]
  [0 4 2 6]]

 [[1 4 2 6]
  [0 5 2 6]
  [1 5 2 6]]

 [[0 3 4 4]
  [1 3 4 4]
  [2 3 4 4]]

 [[3 3 4 4]
  [0 0 0 0]
  [0 0 0 0]]]


Finally, see if you can use what you have learned to create kernel that multiplies the elements in matrix a by matrix b. The outline has been provided. Please see if you can complete the code and produce a correct result!
The assertion will fail if you don't. Remember to edit the launch configuration.

Compare to serial version 

 - in python?
 
 - with a 1x1x1 grid?
 

In [0]:
mmult = SourceModule("""
  __global__ void multiply(int *a, int *b)
  {
    // initialise indices
    int idx = 4 * (blockIdx.x * blockDim.x + threadIdx.x);
    
    
    int a_idx = 4 * (blockIdx.x * blockDim.x + threadIdx.x);
    int b_idx = 4 * (blockIdx.x * blockDim.x + threadIdx.x);
    // initialise output matrix
    int ab = 4 * (blockIdx.x * blockDim.x + threadIdx.x);
    // multiply each element of a by b 
    
    
    //multiply
    
    // vector for each element of AB
    
    //sum each element
    ab[idx] = a[a_idx] * a[a_idx];
    
    
    int idx = threadIdx.x;
    for (int i=0;i<16;i+=4)
    {
      a[idx + i] = idx + i;
    }
    
   
    // TODO
  }
  """)

a = np.array([[1,2,3,4],[1,2,3,4],[1,2,3,4],[1,2,3,4]],dtype=np.uint32)
b = np.array([[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]],dtype=np.uint32)

# Allocate GPU memory
a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
# set the gpu memory
cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)

# Get the function handle
func = mmult.get_function("multiply")
# Create a lauch configuration with 4 threads and 1 block
func(a_gpu, b_gpu, TODO ......)

# Create array for output
a_out = np.empty_like(a)
# Copy the gpu memory to the host
cuda.memcpy_dtoh(a_out, a_gpu)

# Print data out
print(a_out)

# Free up the memory
a_gpu.free()
b_gpu.free()

assert (a_out == [[1,2,3,4],[2,4,6,8],[3,6,9,12],[4,8,12,16]]).all()


There is a simpler way of handling arrays than what we have so far looked at. The GPUArray is an abstraction that removes some of the steps. It is useful to understand what those steps did though, because it helps your understanding of what is going on, which is helpful when it comes to getting the best performance from your code.

In [0]:
from pycuda import gpuarray

x = np.random.randn(5).astype(np.float32)
x_gpu = gpuarray.to_gpu(x)

x_gpu

You can operate directly on gpuarrays as if they were normal numpy arrays.


In [0]:
x_gpu = x_gpu * 3
x_gpu

GPUArrays also support slicing. The one thing that you cannot do is indexing though:

In [0]:
x_gpu[:3]

x_gpu[3]

In order to use a GPUArray in a function created with SourceModule (as per the examples above) we need to use the **gpudata** attribute:

In [0]:
 func = mod.get_function("doublify")
 func(x_gpu.gpudata, block = (5,1,1))

 x_gpu

As you can see, this is much cleaner than the previous method! 

Next we need to explain a couple of important concepts that, so far, we ignored.
First of all, you may have noticed that the kernel function we declared was preceeded by the \_\_global\_\_ keyword. This tells the compiler that this is kernel (i.e. it will be called from the CPU but executed on the GPU). You will also have seen that the return type was 'void'. This is because kernel function cannot return values. Instead, any results need to be handled by passing in a pointer to a variable and manipulating the data within the kernel.

Additionally, sometimes you need to call a function from within your kernel. This can be done by declaring a \_\_device\_\_ function, which will reside only on the GPU.

Let's create a simple example of a device function:


In [0]:
dev = SourceModule("""
__device__ int times2( float px); // declaration
  __global__ void  matAdd(float *a)  
{  
        int idx = blockIdx.x*blockDim.x+threadIdx.x;  
        
        a[idx] = times2(idx);   

}  
  __device__ int times2( float idx) // implementation
{
    return idx * 2; 
}  
""")

func = dev.get_function("matAdd")
func(x_gpu.gpudata, block = (5,1,1))

x_gpu

Okay, next see if you can create your own matrix multiplication kernel. Remember that there is a reduction operation in a matrix multiplication that needs to be run after the multiplication, so watch out for data races!

In [0]:
# Create your kernel here

In [0]:
# Create your calling code here

Well done (or cmomiserations)!

You have completed the introduction.